# Baseline Architecture
* test qdanalysis package functions
* rough out ML process

In [2]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

from qdanalysis.preprocessing import strokedecomposition as sd

ModuleNotFoundError: No module named 'qdanalysis'